<a href="https://colab.research.google.com/github/Que1Pereza2/Mr.CrabsAnalyzer/blob/main/CanYouFeelItNow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports block

In [46]:
import numpy as np
import pandas as pd
import re
import nltk
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.under_sampling import RandomUnderSampler

This block reads the None2775.csv file and creates the arrays features and labels.

In [47]:
reviews = pd.read_csv("None2775.csv")

features = reviews.iloc[:,0].values
labels = reviews.iloc[:,1].values

This function uses regex to clean the data and prepares it for the neural network to train on.

In [48]:
processed_features = []
for sentence in range(0, len(features)):
    # Remove all the special characters
    processed_feature = re.sub(r'\W', ' ', str(features[sentence]))

    # remove all single characters
    processed_feature= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)

    # Remove single characters from the start
    processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature)

    # Substituting multiple spaces with single space
    processed_feature = re.sub(r'\s+', ' ', processed_feature, flags=re.I)

    # Removing prefixed 'b'
    processed_feature = re.sub(r'^b\s+', '', processed_feature)

    # Converting to Lowercase
    processed_feature = processed_feature.lower()

    processed_features.append(processed_feature)

vectorizer = TfidfVectorizer (max_features=2500, min_df=7, max_df=0.8)
processed_features = vectorizer.fit_transform(processed_features).toarray()


Creating and feeding the data to the Neural Network

In [49]:
rus = RandomUnderSampler(random_state=42, sampling_strategy = 'majority')

X_train, X_test, y_train, y_test = train_test_split(processed_features, labels, test_size=0.2, random_state=0)

X_resampled, y_X_resampled = rus.fit_resample(X_train, y_train)

text_classifier = RandomForestClassifier(criterion="entropy",n_estimators=200, random_state=42)

text_classifier.fit(X_resampled, y_X_resampled)
# text_classifier.fit(X_train, y_train)

predictions = text_classifier.predict(X_test)

from pprint import pprint

# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(text_classifier.get_params())

Parameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'entropy',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 200,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [50]:
from sklearn.metrics import roc_curve, auc
n_estimators = [1, 2, 4, 8, 16, 32, 64, 100, 200]
train_results = []
test_results = []
for estimator in n_estimators:
   rf = RandomForestClassifier(n_estimators=estimator, n_jobs=-1)
   rf.fit(X_resampled, y_X_resampled)
   train_pred = rf.predict(X_resampled)
   false_positive_rate, true_positive_rate, thresholds = roc_curve(y_X_resampled, train_pred)
   roc_auc = auc(false_positive_rate, true_positive_rate)
   train_results.append(roc_auc)
   y_pred = rf.predict(X_resampled)
   false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
   roc_auc = auc(false_positive_rate, true_positive_rate)
   test_results.append(roc_auc)
   from matplotlib.legend_handler import HandlerLine2D
   line1, = plt.plot(n_estimators, train_results, 'b', label="Train AUC")
   line2, = plt.plot(n_estimators, test_results, 'r', label="Test AUC")
   plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})

plt.ylabel('AUC score')
plt.xlabel('n_estimators')
plt.show()

ValueError: y_true takes value in {'"0"', '"1"'} and pos_label is not specified: either make y_true take value in {0, 1} or {-1, 1} or pass pos_label explicitly.

Neural Network stats

In [51]:
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))

[[ 444  101]
 [ 279 1187]]
              precision    recall  f1-score   support

         "0"       0.61      0.81      0.70       545
         "1"       0.92      0.81      0.86      1466

    accuracy                           0.81      2011
   macro avg       0.77      0.81      0.78      2011
weighted avg       0.84      0.81      0.82      2011

0.8110392839383391
